In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

#신경망 가중치와 결과의 무작위성을 줄이기 위해 랜덤 시드를 설정
#결과를 완전히 재현하는 것은 여전히 어려울 수 있음

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# combined_faces 압축 파일 풀기

combined_faces_zip_path = "/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/1.1_age_input_output/input/combined_faces.zip"
extracted_path='/content/combined_faces'
with ZipFile(combined_faces_zip_path, 'r') as myzip:
    myzip.extractall(extracted_path)
    print('Done unzipping combined_faces.zip')

Done unzipping combined_faces.zip


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# 기존에 학습한 모델을 불러옵니다.
model = load_model('/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/1.1_age_input_output/output/100epoch_age_model_acc_0.882.h5')



엑셀파일 만들기!!

In [ ]:
ages = []

path = '/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/change_brightness_UTK'
i = 0

for img_filename in os.listdir(path):
    i = i + 1
    age_gender_filename = img_filename.split("_")
     # Ensure that the file is an image file
    if not img_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    ages.append(int(age_gender_filename[0]))  # 이미지 파일 이름에서 나이 정보 추출

print(ages)

In [ ]:
import os
import pandas as pd

# 사용자가 설정할 열 이름
column_name = 'filename'
column_name2 = 'age'

# 사진이 저장된 디렉토리 경로
photo_directory = '/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/change_brightness_UTK'

# 디렉토리에서 파일 목록 가져오기
photo_files = [f for f in os.listdir(photo_directory) if os.path.isfile(os.path.join(photo_directory, f))]

# 파일 목록을 DataFrame으로 변환
data = {column_name: photo_files}
df = pd.DataFrame(data)

# 추가할 문자열
additional_string = '/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/change_brightness_UTK/'

# 특정 열에 같은 문자열 추가
df[column_name] =additional_string+ df[column_name]

# age 관련 열 추가
df[column_name2] = ages

# DataFrame을 엑셀 파일로 저장
csv_filename = '/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/brightness_UTK.csv'
df.to_csv(csv_filename, index=False)

print(f"엑셀 파일이 생성되었습니다: {csv_filename}")


엑셀 파일이 생성되었습니다: /content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/brightness_UTK.csv


In [ ]:
# 증강된 데이터셋 블러와서 텐서 만들기

train_aug_df = pd.read_csv("/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/brightness_UTK.csv")
test_df = pd.read_csv("/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/1.1_age_input_output/input/images_filenames_labels_test_1.csv")

In [ ]:
# 나이 범주별 라벨링

def class_labels_reassign(age):

    if 1 <= age <= 5:
        return 0
    elif 6<= age <= 10:
        return 1
    elif 11 <= age <= 15:
        return 2
    elif 16 <= age <=20:
        return 3
    elif 21 <= age <= 26:
        return 4
    elif 27 <= age <= 32:
        return 5
    elif 33 <= age <= 39:
        return 6
    elif 40 <= age <= 48:
        return 7
    elif 49 <= age <= 56:
        return 8
    elif 57 <= age <= 65:
        return 9
    else:
        return 10

In [ ]:
train_aug_df['target'] = train_aug_df['age'].map(class_labels_reassign)
test_df['target'] = test_df['age'].map(class_labels_reassign)

In [ ]:
train_aug_df.head


<bound method NDFrame.head of                                               filename  age  target
0    /content/drive/MyDrive/GraduationCapston/Forei...  100      10
1    /content/drive/MyDrive/GraduationCapston/Forei...   57       9
2    /content/drive/MyDrive/GraduationCapston/Forei...   13       2
3    /content/drive/MyDrive/GraduationCapston/Forei...   29       5
4    /content/drive/MyDrive/GraduationCapston/Forei...   57       9
..                                                 ...  ...     ...
956  /content/drive/MyDrive/GraduationCapston/Forei...    1       0
957  /content/drive/MyDrive/GraduationCapston/Forei...   45       7
958  /content/drive/MyDrive/GraduationCapston/Forei...   33       6
959  /content/drive/MyDrive/GraduationCapston/Forei...   38       6
960  /content/drive/MyDrive/GraduationCapston/Forei...   24       4

[961 rows x 3 columns]>

In [ ]:
#이미지 크기 확인

import cv2
import os


def get_image_size(image_path):
    # 이미지를 읽어들임
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    if img is not None:
        # 이미지의 높이와 너비를 반환
        height, width = img.shape[:2]
        return height, width
    else:
        return None

# 이미지 파일이 들어있는 디렉토리 경로
image_directory = '/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/change_brightness_UTK'

# 디렉토리에서 파일 목록 가져오기
image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

# 각 이미지의 크기를 확인
for image_file in image_files:
    image_path = os.path.join(image_directory, image_file)
    size = get_image_size(image_path)

    if size is not None:
        print(f"Image: {image_file}, Size: {size}")
    else:
        print(f"Error reading image: {image_file}")


Image: 100_1_0_20170112213001988.jpg.chip.jpg, Size: (200, 200)
Image: 57_0_3_20170119211458431.jpg.chip.jpg, Size: (200, 200)
Image: 13_0_0_20170110232526929.jpg.chip.jpg, Size: (200, 200)
Image: 29_1_1_20170114024847193.jpg.chip.jpg, Size: (200, 200)
Image: 57_1_0_20170104205902805.jpg.chip.jpg, Size: (200, 200)
Image: 52_0_0_20170109003616600.jpg.chip.jpg, Size: (200, 200)
Image: 20_1_4_20170103223401848.jpg.chip.jpg, Size: (200, 200)
Image: 37_0_0_20170109013431279.jpg.chip.jpg, Size: (200, 200)
Image: 26_0_3_20170119180344516.jpg.chip.jpg, Size: (200, 200)
Image: 41_0_4_20170104173222571.jpg.chip.jpg, Size: (200, 200)


KeyboardInterrupt: ignored

In [ ]:
# 증강된 훈련,검증 데이터셋들을 위해 파일 이름과 타겟 클래스 라벨 변경

train_aug_filenames_list = list(train_aug_df['filename'])
train_aug_labels_list = list(train_aug_df['target'])

test_filenames_list = list(test_df['filename'])
test_labels_list = list(test_df['target'])

In [ ]:
# 위에서 정의한 목록들을 사용하여 증강된(데이터를 늘린) 학습 및 테스트 데이터셋에 대한 TensorFlow 상수를 만들고, 파일 이름과 레이블을 나타내기

train_aug_filenames_tensor = tf.constant(train_aug_filenames_list)
train_aug_labels_tensor = tf.constant(train_aug_labels_list)

test_filenames_tensor = tf.constant(test_filenames_list)
test_labels_tensor = tf.constant(test_labels_list)

In [ ]:
# 주어진 텐서로부터 이미지를 읽고, 이미지를 해독하며, 이미지 레이블 클래스를 원-핫 인코딩하는 함수 정의.
# tf.io.decode_jpeg에서 channels 매개변수를 3에서 1로 변경하면 출력 이미지가 RGB 색상이 아닌 그레이스케일로 변함

num_classes = 11

def _parse_function(filename, label):

    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=1)    # channels=1 그레이 스케일로 바꾸기 위해서, channels=3 RGB로 바꾸기 위해서.
    # image_resized = tf.image.resize(image_decoded, [200, 200])
    label = tf.one_hot(label, num_classes)

    return image_decoded, label

In [ ]:
# 뉴런 네트워크를 위해 데이터셋 가져오기
# 위에서 정의한 텐서 벡터를 사용하여 데이터셋에서 이미지에 접근하고, 위에서 정의한 함수를 통해 이미지를 처리

train_aug_dataset = tf.data.Dataset.from_tensor_slices((train_aug_filenames_tensor, train_aug_labels_tensor))
train_aug_dataset = train_aug_dataset.map(_parse_function)
#train_aug_dataset = train_aug_dataset.repeat(3)
train_aug_dataset = train_aug_dataset.batch(512)    # 아래 model.fit()에서의 batch_size 하이퍼파라미터와 동일

test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames_tensor, test_labels_tensor))
test_dataset = test_dataset.map(_parse_function)
#test_dataset = test_dataset.repeat(3)
test_dataset = test_dataset.batch(512)    # 아래 model.fit()에서의 batch_size 하이퍼파라미터와 동일

In [ ]:

# 모델을 계속해서 학습시킵니다.
# 이때, 기존의 모델의 가중치를 그대로 사용하므로 fine-tuning이라고도 합니다.
model.fit(train_aug_dataset, validation_data=test_dataset, epochs=10)

# 학습이 완료된 모델을 저장합니다.
model.save('/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/1.1_age_input_output/output/your_updated_model.h5')

NameError: ignored

In [ ]:
# 학습이 완료된 모델을 저장합니다.
model.save('/content/drive/MyDrive/GraduationCapston/Foreigner_age_predict/age/1.1_age_input_output/output/your_updated_model.h5')